## Burgers 2D Mapped Tent Pitching and Picard Propagation

In [1]:
from netgen.geom2d import SplineGeometry
from ngsolve import Mesh, CoefficientFunction, x, y, exp
from ngsolve import TaskManager, SetNumThreads
from ngsolve.webgui import Draw
from ngstents import TentSlab, Burgers
import tentswebgui

In [2]:
SetNumThreads(1)
maxh = 0.05
geom = SplineGeometry()
geom.AddRectangle((0, 0), (1, 1), bc=1)
mesh = Mesh(geom.GenerateMesh(maxh=maxh))



In [3]:
heapsize = 10*1000*1000
dt = 0.05   # tent slab height (in time)
c = 16      # wavespeed
ts = TentSlab(mesh, dt, c, heapsize)
print("max slope", ts.MaxSlope())  # need max slope <= 1/wavespeed for causality

tentswebgui.Draw(ts)

max slope 0.09003623964159489
setup: 0.3488192558288574, tents: 0.7289445400238037 faces: 6.151760101318359


NGSTentsWebGuiWidget(value={'ntents': 7923, 'nlayers': 76, 'ngsolve_version': '6.2.2006-26-gf084b4f7', 'tent_e…

In [6]:
order = 4
cf = CoefficientFunction(exp(-50*((x-0.3)*(x-0.3)+(y-0.3)*(y-0.3))))

burg = Burgers(ts, order=order)
burg.SetInitial(cf)
sol = burg.sol
scene = Draw(sol, animate=True)

tend = 3*dt
t = 0
cnt = 0

with TaskManager():
    while t < tend - dt/2:
        burg.PropagatePicard(sol.vec, steps=order*order)
        t += dt
        cnt += 1
        if cnt % 1 == 0:
            print("{:.3f}".format(t))
            scene.Redraw()


NGSWebGuiWidget(value={'ngsolve_version': '6.2.2006-26-gf084b4f7', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, …

0.050
0.100
0.150
